In [ ]:
! sudo pip install snowflake-ml-python==1.0.2 -U 
! pip install snowflake-snowpark-python==1.5.1 -U

     |████████████████████████████████| 1.9MB 8.0MB/s eta 0:00:01


  Found existing installation: snowflake-ml-python 1.0.5
    Uninstalling snowflake-ml-python-1.0.5:
      Successfully uninstalled snowflake-ml-python-1.0.5
  Using cached https://files.pythonhosted.org/packages/a0/4b/0e976c43a24b9d52cdf765d70638121a5b7b76bd651e91240adcdd1d4dd4/snowflake_snowpark_python-1.5.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b8/8b/31273bf66016be6ad22bb7345c37ff350276cfd46e389a0c2ac5da9d9073/wheel-0.41.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/24/21/7d397a4b7934ff4028987914ac1044d3b7d52712f30e2ac7a2ae5bc86dd0/typing_extensions-4.8.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a6/0b/56310babca5020a2c875b294653b2e637be81d7f05b418c9823cd2459644/snowflake_connector_python-3.2.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/bb/26/7945080113158354380a12ce26873dd6c1ebd88d47f5bc24e2c5bb38c16a/setuptools-68.2.2-py3-

In [2]:
! pip install --upgrade xgboost==1.7.3

     |████████████████████████████████| 193.6MB 12kB/s /s eta 0:00:01
     |████████████████████████████████| 17.3MB 93.6MB/s eta 0:00:01
     |████████████████████████████████| 34.5MB 103.2MB/s eta 0:00:01


In [1]:
! pip list

Package                    Version     
-------------------------- ------------
absl-py                    1.4.0       
aiohttp                    3.8.5       
aiosignal                  1.3.1       
alembic                    1.11.1      
anyio                      3.7.0       
argon2-cffi                21.3.0      
argon2-cffi-bindings       21.2.0      
asn1crypto                 1.5.1       
asttokens                  2.2.1       
async-generator            1.10        
async-timeout              4.0.3       
attrs                      23.1.0      
Babel                      2.12.1      
backcall                   0.2.0       
beautifulsoup4             4.12.2      
bleach                     4.1.0       
boltons                    23.0.0      
bqplot                     0.12.39     
brotlipy                   0.7.0       
certifi                    2023.7.22   
certipy                    0.1.3       
cffi                       1.15.1      
charset-normalizer         3.2.0       


In [2]:
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.linear_model import SGDClassifier
from snowflake.ml.modeling.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.metrics import mean_absolute_percentage_error

In [3]:
from snowflake.snowpark import Session
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])

    session = Session.builder.configs(connection_params).create()
    return session



In [4]:
session = get_session()

# Model Building
### Load the final data from snowflake table

In [5]:
table_name = 'predictive_maintenance_final'

In [6]:
sf_df = session.table(table_name).drop('ROW')

In [7]:
sf_df.show(3)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FAILURE"  |"METRIC5"  |"METRIC6"  |"DAY"  |"DAY_WEEK"  |"SECTOR"  |"OP_PERIOD"  |"DEV_RECONNECTED"  |"MNW1"    |"DIF_M6"  |"DIF_M5"  |"LOG_M2"           |"LOG_M3"            |"LOG_M4"            |"LOG_M7"  |"LOG_M9"           |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0          |6          |407438     |1      |3           |S1F0      |1            |0                  |26953834  |0         |0         |2.079441541679836  |0.0                 |3.9702919135521215  |0.0       |2.079441541679836  |
|0          |6          |403174     |1      |3           |S1F0      |1          

In [8]:
a = sf_df.to_pandas()

In [11]:
list(a.columns)

['FAILURE',
 'METRIC5',
 'METRIC6',
 'DAY',
 'DAY_WEEK',
 'SECTOR',
 'OP_PERIOD',
 'DEV_RECONNECTED',
 'MNW1',
 'DIF_M6',
 'DIF_M5',
 'LOG_M2',
 'LOG_M3',
 'LOG_M4',
 'LOG_M7',
 'LOG_M9']

In [10]:
a.head()

,FAILURE,METRIC5,METRIC6,DAY,DAY_WEEK,SECTOR,OP_PERIOD,DEV_RECONNECTED,MNW1,DIF_M6,DIF_M5,LOG_M2,LOG_M3,LOG_M4,LOG_M7,LOG_M9
0,0,6,407438,1,3,S1F0,1,0,26953834,0,0,2.079442,0.000000,3.970292,0.0,2.079442
1,0,6,403174,1,3,S1F0,1,0,7671335,0,0,0.000000,1.386294,0.000000,0.0,0.000000
2,0,12,237394,1,3,S1F0,1,0,21661996,0,0,0.000000,0.000000,0.000000,0.0,0.000000
3,0,6,410186,1,3,S1F0,1,0,9961753,0,0,0.000000,0.000000,0.000000,0.0,0.000000
4,0,15,313173,1,3,S1F0,1,0,16996310,0,0,0.000000,0.000000,0.000000,0.0,1.386294


In [8]:
CATEGORICAL_COLUMNS = ["SECTOR"]
CATEGORICAL_COLUMNS_OE = ["SECTOR_OE"]
NUMERICAL_COLUMNS = ['METRIC5','METRIC6','DAY','DAY_WEEK','OP_PERIOD','MNW1','DIF_M6','DIF_M5','LOG_M2',
 'LOG_M3','LOG_M4','LOG_M7','LOG_M9']
LABEL_COLUMNS = ["FAILURE"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [9]:
train_df, test_df = sf_df.random_split([0.8,0.2])

In [10]:
train_df.show(3)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FAILURE"  |"METRIC5"  |"METRIC6"  |"DAY"  |"DAY_WEEK"  |"SECTOR"  |"OP_PERIOD"  |"DEV_RECONNECTED"  |"MNW1"    |"DIF_M6"  |"DIF_M5"  |"LOG_M2"  |"LOG_M3"            |"LOG_M4"  |"LOG_M7"  |"LOG_M9"            |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0          |6          |403174     |1      |3           |S1F0      |1            |0                  |7671335   |0         |0         |0.0       |1.3862943611198906  |0.0       |0.0       |0.0                 |
|0          |12         |237394     |1      |3           |S1F0      |1            |0                  |21661996  |0         |0         |0.0       |0.0  

Create a pipeline with preprocessing steps and model definition

In [11]:
pipeline = Pipeline(
    steps=[
            (
                "OHE",
                OrdinalEncoder(
                input_cols=CATEGORICAL_COLUMNS,
                output_cols=CATEGORICAL_COLUMNS_OE
                )
            ),
            (
                "classification",
                SGDClassifier(
                input_cols=NUMERICAL_COLUMNS + CATEGORICAL_COLUMNS_OE + ['DEV_RECONNECTED'],
                label_cols=LABEL_COLUMNS,
                output_cols=OUTPUT_COLUMNS
                )
            )
    ]
)

In [12]:
pipeline.fit(train_df)

The version of package numpy in the local environment is 1.24.4, which does not fit the criteria for the requirement numpy==1.24.3. Your UDF might not work when the package version is different between the server and your local environment


In [13]:
result = pipeline.predict(test_df)

The version of package numpy in the local environment is 1.24.4, which does not fit the criteria for the requirement numpy==1.24.3. Your UDF might not work when the package version is different between the server and your local environment


In [14]:
result.show()

SnowparkSQLException: (1304): 01af1b92-0503-cace-0072-f3030bf47366: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "/home/udf/32355341555882230/udf_py_484651959.zip/udf_py_484651959.py", line 110, in compute
    return lock_function_once(func, invoked)(*[df[idx] for idx in range(df.shape[1])])
  File "/home/udf/32355341555882230/udf_py_484651959.zip/udf_py_484651959.py", line 99, in wrapper
    result = f(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/snowflake/ml/modeling/linear_model/sgd_classifier.py", line 731, in vec_batch_infer
AttributeError: module 'pandas.io.json' has no attribute 'json_normalize'
 in function SNOWML_BATCH_INFERENCE_D1B9F470_DCD3_42BB_B8B4_94BB153CE18B_PREDICT with handler udf_py_484651959.compute

In [25]:
sf_df1 = session.table("BANK_CUSTOMER_CHURN_PREDICTION").drop('ROW')

In [26]:
CATEGORICAL_COLUMNS = ["COUNTRY", "GENDER"]
CATEGORICAL_COLUMNS_OE = ["COUNTRY_OE", "GENDER_OE"]
NUMERICAL_COLUMNS = ["CREDIT_SCORE", "AGE", "TENURE", "BALANCE", "PRODUCTS_NUMBER", "ESTIMATED_SALARY"]
LABEL_COLUMNS = ["CHURN"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [27]:
train_df, test_df = sf_df.random_split([0.8,0.2])

In [28]:
pipeline = Pipeline(
    steps=[
            (
                "OE",
                OrdinalEncoder(
                input_cols=CATEGORICAL_COLUMNS,
                output_cols=CATEGORICAL_COLUMNS_OE
                )
            ),
            (
                "MMS",
                MinMaxScaler(
                clip=True,
                input_cols=NUMERICAL_COLUMNS,
                output_cols=NUMERICAL_COLUMNS,
                )
            ),
            (
                "classification",
                XGBClassifier(
                input_cols=CATEGORICAL_COLUMNS_OE+NUMERICAL_COLUMNS + ["CREDIT_CARD"],
                label_cols=LABEL_COLUMNS,
                output_cols=OUTPUT_COLUMNS
                )
            )
    ]
)

In [29]:
pipeline.fit(train_df)

The version of package 'numpy' in the local environment is 1.24.4, which does not fit the criteria for the requirement 'numpy==1.24.3'. Your UDF might not work when the package version is different between the server and your local environment.


In [30]:
result = pipeline.predict(test_df)

The version of package 'numpy' in the local environment is 1.24.4, which does not fit the criteria for the requirement 'numpy==1.24.3'. Your UDF might not work when the package version is different between the server and your local environment.


In [31]:
result.show()

SnowparkSQLException: (1304): 01af1b8a-0503-cace-0072-f3030bf470b2: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "/home/udf/32355341555882206/udf_py_1266416964.zip/udf_py_1266416964.py", line 110, in compute
    return lock_function_once(func, invoked)(*[df[idx] for idx in range(df.shape[1])])
  File "/home/udf/32355341555882206/udf_py_1266416964.zip/udf_py_1266416964.py", line 99, in wrapper
    result = f(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/snowflake/ml/modeling/xgboost/xgb_classifier.py", line 748, in vec_batch_infer
AttributeError: module 'pandas.io.json' has no attribute 'json_normalize'
 in function SNOWML_BATCH_INFERENCE_4DA7A050_5BDD_404F_ABF7_E6283071B336_PREDICT with handler udf_py_1266416964.compute